**jumbo rolls are folded in**

**changeover between jumbos are folded in**

**Minimize knife changes folded in**

need to ..

* show when an order in the future has been fullfilled.

* look forward in schedule to optimize new/better layouts

* make layout table editable

* speed test algorithm

* LM get rid of decimals

check work orders to full jumbo

other 

* student help

# Simple Genetic Algorithm to Optimize Deckle Loss

In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from genetic import *
from IPython.display import clear_output
import time
import datetime
import re

In [3]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
temp = df_input_schedule[['Customer Name', 'Technology', 'Color', 'Width', 'CYCLE / BUCKET',
                          'Description', 'Total LM Order QTY',
                          'Scheduled Ship Date', 'Date order is complete']]

In [4]:
# temp = temp[temp['Scheduled Ship Date'].apply(lambda x: type(x) == int)]                          
temp_json = temp.to_json()

In [5]:
temp2 = pd.read_json(temp_json, 
                     convert_dates=['Scheduled Ship Date', 'Date order is complete'])

## to_datetime and to_json use default units ms and ns, respectively
temp2['Scheduled Ship Date'] = pd.to_datetime(temp2['Scheduled Ship Date'], 
                                              errors='ignore', unit='ms')

In [6]:
### not currently used, grabs indicated information
customer_names = [i for i in df['Customer Name'].unique() if type(i) == str]
technologies = list(set([parse_description(i, 'tech') for i in df['Description'] 
                         if (type(i) == str)]))
technologies = [i for i in technologies if i]
colors = list(set([i[1] for i in list(df['Description']
                .str.split(';')) if type(i) == list]))
cycles = list(df['CYCLE / BUCKET'].unique())

In [7]:
customer = 'P & G'
technology = 'SAM' # Technology -> 
color = 'WHITE'
cycle = 'CYCLE 2'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['Order Number'] = df_filtered.index + 1

df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]

In [8]:
### Detects if there is a break in the rolls coming of the
### line

prev_index = df_filtered.index[0] - 1
block = 1
for index in df_filtered.index:
    if index != prev_index + 1:
        block += 1
    df_filtered.loc[index, 'Block'] = block
    prev_index = index

In [9]:

lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
w = list(np.array(widths) + np.array(neckin)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckin))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [154, 158, 170, 208, 218, 234] (mm)
neck in: [4, 4, 5, 7, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [96, 32, 318, 95, 158, 461]


In [10]:
# customer = 'P & G'
# product = 'SAM' # Technology -> 
# color = 'WHITE'
# cycle = 'CYCLE 2'

# df = pd.read_excel('../data/berry/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
#                    sheet_name='Schedule')
# df = df.loc[df['Customer Name'] == customer]
# df = df.loc[df['Description'].str.contains(product)]
# df = df.loc[df['Description'].str.contains(color)] #CYAN, TEAL
# df = df.loc[df['CYCLE / BUCKET'] == cycle]
# df = df.reset_index(drop=True)
# df['Width'] = pd.DataFrame(list(pd.DataFrame(list(df['Description']
#                                                   .str.split(';')))[0].str.split('UN0')))[1]
# lm = list(df.groupby('Description')['Total LM Order QTY'].sum().values)
# widths = list(df.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
# doffs_in_jumbo = 6
# B = 4160
# L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
# w = list(np.array(widths) + np.array(neckin))
# q = [math.ceil(x/L) for x in lm]
# print("max doffs needed without stock cutting: {}".format(q))
# s = BinPackingExample(w, q)
# df_json = df.to_json()

if we gave each order its own entire row we would produce a lot of wasted material

# Run Deckle Optimizer

In [11]:
sol, loss = find_optimum(s, B, widths, neckin, max_unique_products=4, 
                         loss_target=2, max_doff_layouts=10)

gene time: 2.1, chromosome time: 1600456426.4, ffd time: 2.1
loss: 0.962, best loss: 0.962, steps: 9, gene count: 5, unique doffs: 8, max unique on doff: 4
                         Formula Doffs  Loss
0  154x2 + 170x7 + 218x2 + 234x9    45     0
1                         208x19     4    75
2                         218x18     3   110
3                         234x17     3    63
4                 154x6 + 158x11     1  1430
5         158x21 + 170x3 + 208x1     1    18
6                 208x18 + 218x1     1    65
7                 218x13 + 234x5     1    30


# Build Layout for Dashboard View

In [12]:
layout_summary(sol, widths, neckin, B)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,Doffs,Loss
Doffs,,,,,,,,,,,,,,,,,,,,,
1,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1430.0
45,170.0,234.0,218.0,234.0,170.0,170.0,234.0,170.0,154.0,218.0,...,234.0,234.0,154.0,NaN,NaN,NaN,NaN,NaN,45,0.0
1,208.0,170.0,170.0,170.0,158.0,158.0,158.0,158.0,158.0,158.0,...,158.0,158.0,158.0,158.0,158.0,158.0,158.0,158.0,1,18.0
4,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,...,208.0,208.0,NaN,NaN,NaN,NaN,NaN,NaN,4,75.0
1,218.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,208.0,...,208.0,208.0,NaN,NaN,NaN,NaN,NaN,NaN,1,65.0
3,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,...,218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,110.0
1,234.0,234.0,234.0,234.0,234.0,218.0,218.0,218.0,218.0,218.0,...,218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30.0
3,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,63.0


# Run Master Schedule, Optimizing Either for On-Time Orders or Fastest Schedule

In [13]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Product Parameters')

In [14]:
start_date = datetime.datetime(2020,8,1)
start_date

datetime.datetime(2020, 8, 1, 0, 0)

In [15]:
sol, loss = find_optimum(s, B, widths, neckin, max_unique_products=4, 
                         loss_target=2, max_doff_layouts=10)

gene time: 1.4, chromosome time: 1600456428.9, ffd time: 1.4
loss: 0.962, best loss: 0.962, steps: 7, gene count: 5, unique doffs: 10, max unique on doff: 4
                          Formula Doffs Loss
0           170x9 + 218x5 + 234x6    31   14
1                          234x17    16   63
2                          208x19     4   75
3                          154x26     2   52
4                          154x25     1  210
5                  154x19 + 158x7     1   24
6                          158x25     1  110
7                          170x23     1  135
8                  170x15 + 208x7     1   30
9  170x1 + 208x12 + 218x3 + 234x3     1    7


In [16]:
master_knife = optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date, 'Knife Changes')
deckle_time = master_knife.iloc[-1]['Completion Date'] - master_knife.iloc[0]['Completion Date']
print("deckle time, knife changes objective: {}".format(str(deckle_time).split('.')[0]))

../utils.py:490: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




no valid objective provided


AttributeError: 'int' object has no attribute 'iloc'

In [ ]:
master_late = optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date, 'Late Orders')
deckle_time = master_late.iloc[-1]['Completion Date'] - master_late.iloc[0]['Completion Date']
print("deckle time, late orders objective: {}".format(str(deckle_time).split('.')[0]))

In [ ]:
master_knife

In [ ]:
master_late